In [5]:
import folium
import json
import requests
from datetime import datetime
from traveltimepy import TravelTimeSdk, Walking, Coordinates
import asyncio
import os

In [6]:
# list all geojson files in blue_line folder that contain the word 'walking'
paths_walking = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'walking' in fn]


stations_walking = []
# save JSON to variable
for i in range(len(paths_walking)):
    with open(paths_walking[i]) as f:
        data = json.load(f)
        stations_walking.append(data)

# repeat for bike
paths_bike = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'cycling' in fn]

stations_bike = []
for i in range(len(paths_bike)):
    with open(paths_bike[i]) as f:
        data = json.load(f)
        stations_bike.append(data)

In [8]:
lat = 32.86926550131975
lng = -117.21402645890504
map_obj = folium.Map(location=[lat, lng], zoom_start=13)



# add bike in red
fg_bike = folium.FeatureGroup(name='Bike')
for station in stations_bike:
    for i in range(len(station['features'])):
        if "last station: " in station['features'][i]['properties']['search_id']:
            # add to feature group
            folium.GeoJson(station['features'][i], style_function=lambda x: {'color':'red'}).add_to(fg_bike)

# add walking in blue
fg_walking = folium.FeatureGroup(name='Walking')
for station in stations_walking:
    for i in range(len(station['features'])):
        if "last station: " in station['features'][i]['properties']['search_id']:
            # add to feature group
            folium.GeoJson(station['features'][i], style_function=lambda x: {'color':'blue'}).add_to(fg_walking)

# add feature groups to map
map_obj.add_child(fg_bike)
map_obj.add_child(fg_walking)

# add layer control
folium.LayerControl().add_to(map_obj)

map_obj